In [5]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [6]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # train data path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

### Dividing the features by the type of particles: lepton, hadronic tau, jets and missing transverse energy

In [7]:
# dividing the columns of tX by the type of particles
indeces_lepton = [0, 1, 2, 3, 7, 8, 9, 10, 11, 12, 16, 17, 18, 21, 22]
indeces_hadronic_tau = [0, 3, 7, 8, 9, 10, 11, 13, 14, 15, 21, 22]
indeces_jet = [0, 4, 5, 6, 8, 9, 12, 21, 22, 23, 24, 25, 26, 27, 28, 29]
indeces_MTE = [0, 1, 3, 8, 9, 11, 19, 20, 21, 22]

# constructing the submatrices and adding the extra-column of np.ones
tX_lepton = tX[:, indeces_lepton]
tX_tilda_lepton = np.insert(tX_lepton, 0, np.ones(tX.shape[0]), axis=1)
tX_hadronic_tau = tX[:, indeces_hadronic_tau]
tX_tilda_hadronic_tau = np.insert(tX_hadronic_tau, 0, np.ones(tX.shape[0]), axis=1)
tX_jet = tX[:, indeces_jet]
tX_tilda_jet = np.insert(tX_jet, 0, np.ones(tX.shape[0]), axis=1)
tX_MTE = tX[:, indeces_MTE]
tX_tilda_MTE = np.insert(tX_MTE, 0, np.ones(tX.shape[0]), axis=1)

In [8]:
# colors = ['red', 'blue']
# x_pos=[]
# x_neg=[]

# for j in range(len(y)):
#  if(y[j]==1):
#       x_pos.insert(0,tX[j])
#    else:
#        x_neg.insert(0,tX[j])
# xpos = np.array(x_pos)
# xneg = np.array(x_neg)
# for i in range(tX.shape[1]):
#  plt.hist(xpos[:,i], alpha = 0.5, color = 'r', bins = 100)
#  plt.hist(xneg[:,i], alpha = 0.5, color = 'b', bins = 100)
#  plt.show()

## Do your thing crazy machine learning thing here :) ...

In [9]:
def compute_loss(y, tx, w):
    N = y.shape[0]
    e = y - tx @ w 
    loss = 1/(2*N) * np.dot(e,e)
    return loss

In [10]:
def compute_gradient(y, tx, w):
    N = y.shape[0]
    e = y - tx @ w
    gradient = -(1/N) * (tx.T) @ (e)
    return gradient

In [11]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        loss = compute_loss(y,tx,w)
        gradient = compute_gradient(y,tx,w)
        w = w - gamma * gradient
        ws.append(w)
        losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return losses, ws

In [12]:
def compute_stoch_gradient(y, tx, w):
    N = y.shape[0]
    random_number = random.randint(0,N)
    #random_number =1
    xn = tx[random_number,:]
    random_gradient = - np.dot(xn, y[random_number] - np.dot(xn,w))
    return random_gradient

In [13]:
def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        loss = compute_loss(y,tx,w)
        stoch_gradient = compute_stoch_gradient(y,tx,w)
        w = w - gamma * stoch_gradient
        ws.append(w)
        losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return losses, ws

In [14]:
from proj1_helpers import *

def least_squares(y, tx):
    """calculate the least squares solution."""
    forcing_term = np.transpose(tx) @ y
    coefficient_matrix = np.transpose(tx) @ tx
    w = np.linalg.solve(coefficient_matrix, forcing_term)
    return w

def test_your_least_squares(y, tx):
    """compare the solution of the normal equations with the weights returned by gradient descent algorithm."""
    w_least_squares = least_squares(y, tx)
    initial_w = np.zeros(tx.shape[1])
    max_iters = 50
    gamma = 0.7
    losses_gradient_descent, w_gradient_descent = gradient_descent(y, tx, initial_w, max_iters, gamma)
    w = w_gradient_descent[-1]
    err = np.linalg.norm(w_least_squares-w)
    return err

In [15]:
def ridge_regression(y, tx, lambda_):
    """implement ridge regression."""
    N = tx.shape
    lambda_prime = 2 * N[0] * lambda_
    coefficient_matrix = np.transpose(tx) @ tx + lambda_prime * np.eye(N[1])
    forcing_term = np.transpose(tx) @ y
    w = np.linalg.solve(coefficient_matrix, forcing_term)
    return w

def debug_ridge(y, tx):
    """debugging the ridge regression by setting lambda=0."""
    w_least_squares = least_squares(y, tx)
    w_0 = ridge_regression(y, tx, 0)
    err = np.linalg.norm(w_least_squares-w_0)
    return err

In [16]:
def sigmoid(t):
    """apply the sigmoid function on t."""
    return np.exp(t) / (1+np.exp(t))

In [17]:
def calculate_loss(y, tx, w):
    """compute the loss: negative log likelihood."""
    N = y.shape[0]
    e = - (y*np.log(sigmoid(tx @ w)) +
                  (1-y)*np.log(1-sigmoid(tx @ w)))
    return e.sum()

In [18]:
def calculate_gradient(y, tx, w):
    """compute the gradient of loss."""
    return np.transpose(tx) @ (sigmoid(tx @ w) - y)

In [19]:
def learning_by_gradient_descent(y, tx, w, gamma):
    """
    Do one step of gradient descent using logistic regression.
    Return the loss and the updated w.
    """
    loss = calculate_loss(y, tx, w)
    grad = calculate_gradient(y, tx, w)
    w = w - gamma * grad
    return loss, w

In [20]:
def calculate_hessian(y, tx, w):
    """return the Hessian of the loss function."""
    diag = sigmoid(tx @ w) * (1 - sigmoid(tx @ w))
    D = diag * np.eye(tx.shape[0])
    return np.transpose(tx) @ D @ tx

In [21]:
def logistic_regression(y, tx, w):
    """return the loss, gradient, and Hessian."""
    grad = calculate_gradient(y, tx, w)
    hess = calculate_hessian(y, tx, w)
    loss = calculate_loss(y, tx, w)
    return loss, grad, hess

In [22]:
def learning_by_newton_method(y, tx, w, gamma):
    """
    Do one step on Newton's method.
    return the loss and updated w.
    """
    loss, grad, hess = logistic_regression(y, tx, w)
    sol = np.linalg.solve(hess, grad)
    w = w - gamma * sol
    return loss, w

In [23]:
def penalized_logistic_regression(y, tx, w, lambda_):
    """return the loss, gradient"""
    loss = calculate_loss(y, tx, w) + lambda_*np.linalg.norm(w) ** 2
    grad = calculate_gradient(y, tx, w) + 2*lambda_*w
    hess = calculate_hessian(y, tx, w) + 2*lambda_*np.eye(w.shape[0])
    return loss, grad, hess

In [24]:
def learning_by_penalized_gradient(y, tx, w, gamma, lambda_):
    """
    Do one step of gradient descent, using the penalized logistic regression.
    Return the loss and updated w.
    """
    loss, grad, hess = penalized_logistic_regression(y, tx, w, lambda_)
    sol = np.linalg.solve(hess, grad)
    w = w - gamma * sol
    return loss, w

In [25]:
w_opt_lepton = least_squares(y, tX_tilda_lepton)
print(w_opt_lepton, compute_loss(y, tX_tilda_lepton, w_opt_lepton))
w_opt_hadronic_tau = least_squares(y, tX_tilda_hadronic_tau)
print(w_opt_hadronic_tau, compute_loss(y, tX_tilda_hadronic_tau, w_opt_hadronic_tau))
w_opt_jet = least_squares(y, tX_tilda_jet)
print(w_opt_jet, compute_loss(y, tX_tilda_jet, w_opt_jet))
w_opt_MTE = least_squares(y, tX_tilda_MTE)
print(w_opt_MTE, compute_loss(y, tX_tilda_MTE, w_opt_MTE))

[-2.48944227e-01  3.17155005e-05 -5.68780568e-03 -5.03448176e-03
  2.26548943e-03  3.30494490e-01 -1.78706325e-03  1.92770580e-04
 -4.11728751e-01  1.00293148e-01  2.59522112e-04  1.51921403e-02
 -4.88270187e-04  1.54758210e-03 -6.17564602e-04 -7.98573820e-02] 0.3569968339959757
[-5.08332007e-01  2.76131181e-04  2.40191625e-03  1.29950980e-01
 -2.65897946e-03  4.21722672e-04 -1.71144391e-01  1.40854018e-01
  4.59686296e-03 -1.71382443e-03 -1.39788744e-03 -8.43013799e-04
  6.38624491e-03] 0.37908232917533424
[ 3.84625274e-02  4.67823485e-04  1.41475882e-02  4.37437022e-04
 -3.09586633e-02 -3.78256151e-04  2.08599436e-03  4.10891768e-02
 -1.32836024e-04 -2.34820097e-01  2.33056455e-03 -1.41849910e-03
 -6.32284489e-04 -4.19118042e-04 -8.83615251e-03 -1.51544123e-02
 -3.40450916e-03] 0.3964360782407947
[ 0.02257472  0.00022408 -0.0077485  -0.00123365 -0.00230874  0.00161954
  0.11028279  0.0030404   0.00056975 -0.0006231  -0.06961608] 0.3789900059446768


## Generate predictions and save ouput in csv format for submission:

In [184]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [185]:
def predict_labels(weights, tX_test):
    y = np.array(tX_test) @ np.array(weights)
    labels = [1 if l > 0 else -1 for l in y]
    return labels

In [186]:
 #indeces_lepton = [0, 1, 2, 3, 7, 8, 9, 10, 11, 12, 16, 17, 18, 21, 22]
# indeces_hadronic_tau = [0, 3, 7, 8, 9, 10, 11, 13, 14, 15, 21, 22]
# indeces_jet = [0, 4, 5, 6, 8, 9, 12, 21, 22, 23, 24, 25, 26, 27, 28, 29]
# indeces_MTE = [0, 1, 3, 8, 9, 11, 19, 20, 21, 22]

tX_lepton_test = tX_test[:, indeces_lepton]
tX_tilda_lepton_test = np.insert(tX_lepton_test, 0, np.ones(tX_test.shape[0]), axis=1)
labels_lepton = predict_labels(w_opt_lepton, tX_tilda_lepton_test)
print(1 / np.array(labels_lepton).shape[0] * np.count_nonzero(np.array(labels_lepton) == 1))

tX_hadronic_tau_test = tX_test[:, indeces_hadronic_tau]
tX_tilda_hadronic_tau_test = np.insert(tX_hadronic_tau_test, 0, np.ones(tX_test.shape[0]), axis=1)
labels_hadronic_tau = predict_labels(w_opt_hadronic_tau, tX_tilda_hadronic_tau_test)
print(1 / np.array(labels_hadronic_tau).shape[0] * np.count_nonzero(np.array(labels_hadronic_tau) == 1))

tX_jet_test = tX_test[:, indeces_jet]
tX_tilda_jet_test = np.insert(tX_jet_test, 0, np.ones(tX_test.shape[0]), axis=1)
labels_jet = predict_labels(w_opt_jet, tX_tilda_jet_test)
print(1 / np.array(labels_jet).shape[0] * np.count_nonzero(np.array(labels_jet) == 1))

tX_MTE_test = tX_test[:, indeces_MTE]
tX_tilda_MTE_test = np.insert(tX_MTE_test, 0, np.ones(tX_test.shape[0]), axis=1)
labels_MTE = predict_labels(w_opt_MTE, tX_tilda_MTE_test)
print(1 / np.array(labels_MTE).shape[0] * np.count_nonzero(np.array(labels_MTE) == 1))

0.2619606573302032
0.2042559631703617
0.10764151640685769
0.23987660100169295


In [187]:
count = np.array(labels_MTE) + np.array(labels_lepton) + np.array(labels_hadronic_tau) + np.array(labels_jet)
TrueSignal = np.array([int(bool((counted == 4))) for counted in count])
print( 1 / np.array(count).shape[0] * np.count_nonzero(np.array(TrueSignal) == 1))

0.039636560736874334


In [ ]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)